Whenever `YT` returns physical data, it is typically associated with certain units (e.g.,
density in grams per cubic centimeter, temperature in Kelvin, and so on). `YT` exposes the
`YTArray`, `YTQuantity`, and units facilities from `yt` so that "unitful" objects may be
manipulated and operated on. We'll use the following dataset in these examples:

In [ ]:
using YT

ds = load("GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0150")

## Arrays

If we grab the `"density"` field from a sphere, it will be returned as a `YTArray` in $\rm{g}/\rm{cm}^3$:


In [ ]:
sp = Sphere(ds, "c", (100.,"kpc"))

In [ ]:
sp["density"]

A `YTArray` can be manipulated in many of the same ways that normal Julia `Array`s are, and the units are retained. The following are some simple examples of this.

Finding the maximum density:

In [ ]:
maximum(sp["density"])

Multiplying the temperature by a constant unitless number:

In [ ]:
sp["temperature"]*5

Adding two `YTArray`s:

In [ ]:
sp["velocity_magnitude"]+sp["sound_speed"]

Multiplying element-wise one `YTArray` by another:

In [ ]:
sp["density"].*sp["temperature"]

However, attempting to perform an operation that doesn't make sense will throw an error. For example, suppose that you tried to instead add `"density"` and `"temperature"`, which aren't the same type of physical quantity:

In [ ]:
sp["density"]+sp["temperature"]

It is also possible to create a `YTArray` from a regular Julia `Array`, like so:

In [ ]:
a = YTArray(randn(10), "erg")

If your `YTArray` needs to know about code units associated with a specific dataset,
you'll have to create it with a `Dataset` object passed in:

In [ ]:
b = YTArray(ds, [1.0,1.0,1.0], "code_length")

A `YTArray` can be saved to an [HDF5](http://www.hdfgroup.org) file for re-loading later. For this, one can use `write_hdf5()`. One can supply the optional arguments: 

* ``dataset_name``: The name of the dataset to store the array in (defaults to ``"array_data"``)
* ``info``: An optional dictionary which can be stored as dataset attributes to provide additional information

In [ ]:
c = YTArray(rand(10,10), "kpc/Myr")
myinfo = Dict("field"=>"velocity_magnitude", "source"=>"galaxy cluster")
write_hdf5(c, "my_file.h5", dataset_name="cluster", info=myinfo)

The data can be read back into a `YTArray` using `from_hdf5()`:

In [ ]:
d = from_hdf5("my_file.h5", dataset_name="cluster")
c == d # Check to see if they're the same

## Special Arrays

It may be useful to generate `YTArray`s of ones or zeros similar to an existing `YTArray`. This
can be done with `ones` and `zeros`, in the same manner as the standard Julia `Array`:

In [ ]:
ones(sp["density"])

In [ ]:
zeros(d)

To create a `YTArray` by taking a `YTQuantity` (see below) and repeating it, use `fill`:

In [ ]:
q = YTQuantity(200., "nG")
fill(q, 10)

This can be done for multi-dimensional arrays as well:

In [ ]:
fill(q, (10,10))

If you have a 2D `YTArray` and would like to create an identity matrix with the same shape, use `eye`:

In [ ]:
Q = YTArray(rand(5,5), "kC")
eye(Q)

## Quantities

A `YTQuantity` is just a scalar version of a `YTArray`. They can be created and manipulated in much the same way:

In [ ]:
theta = YTQuantity(3.14159, "radian")

In [ ]:
l = YTQuantity(12, "cm")

In [ ]:
theta/l

In [ ]:
theta\l

In [ ]:
p = YTQuantity(13,"m")
l+p

In [ ]:
f = YTQuantity(ds, 2.5, "code_length")

## Changing Units

Occasionally you will want to change the units of an array or quantity to something more appropriate. Taking density as the example, we can change it to units of solar masses per kiloparsec, using `convert_to_units()`:

In [ ]:
convert_to_units(sp["density"], "Msun/kpc^3")

In [ ]:
sp["density"]

There are also special methods for conversions to cgs and MKS unit systems:

In [ ]:
convert_to_cgs(sp["density"])
sp["density"]

In [ ]:
convert_to_mks(sp["density"])
sp["density"]

The above do in-place conversions of the original array or quantity. To create a new array or quantity from a unit conversion of an existing one, use the `in_units()`, `in_cgs()`, and `in_mks()` methods, which have the same signature, and return the new array or quantity:

In [ ]:
in_units(sp["density"], "Msun/kpc^3")

We can check to see that this array is where we left it:

In [ ]:
sp["density"]

## Unit Systems

yt and `YT` come with a number of built-in unit systems. You have already seen two of them, `"cgs"` and `"mks"`. There are others. The full set includes:

* `"cgs"`: Centimeters-grams-seconds unit system, with base of `(cm, g, s, K, radian)`. Uses the Gaussian normalization for electromagnetic units.
* `"mks"`: Meters-kilograms-seconds unit system, with base of `(m, kg, s, K, radian, A)`.
* `"imperial"`: Imperial unit system, with base of `(mile, lbm, s, R, radian)`.
* `"galactic"`: "Galactic" unit system, with base of `(kpc, Msun, Myr, K, radian)`.
* `"solar"`: "Solar" unit system, with base of `(AU, Mearth, yr, K, radian)`.
* `"planck"`: Planck natural units $\hbar = c = G = k_B = 1$, with base of `(l_pl, m_pl, t_pl, T_pl, radian)`.
* `"geometrized"`: Geometrized natural units $c = G = 1$, with base of `(l_geom, m_geom, t_geom, K, radian)`.

There is a `unit_system_registry` `Dict` that can be queried for the different unit systems:

In [ ]:
collect(keys(unit_system_registry))

A particular registry can also be queried to determine what units it uses for a particular dimension:

In [ ]:
unit_system_registry["mks"]

In [ ]:
unit_system_registry["mks"]["time"]

Any given `YTArray` or `YTQuantity` can be converted to a different unit system using the `convert_to_base()` and `in_base()` methods:

In [ ]:
aa = YTArray(rand(10), "m")
in_base(aa; unit_system="imperial")

## Mathematical Functions and Array Methods

A number of standard mathematical functions and array methods in Julia work on `YTArray`s:

* `sqrt`: (square root)
* `abs`:  (absolute value)
* `abs2`: (square of the absolute value)
* `minimum`: (minimum of an array)
* `maximum`: (maximum of an array)
* `hypot`: (square root of the sum of squares)
* `size`: (size of an array)
* `ndims`: (number of dimensions of an array)
* `sum`, `sum_kbn`: (sum of array elements)
* `cumsum`, `cumsum_kbn`: (cumulative sum of array elements)
* `accumulate`: (accumulation of array elements, such as sum, max, min, etc.)
* `diff`: (finite difference operator of an array)
* `gradient`: (differences along an array with a specified spacing between points)
* `mean`: (arithmetic mean of an array)
* `std`, `stdm`: (standard deviation of an array)
* `var`, `varm`: (variance of an array)
* `midpoints`: (midpoints of array)
* `median`: (median of an array)
* `middle`: (middle of an array or two numbers)
* `quantile`: (quantile(s) of an array)

For more information on how these methods work in Julia, please consult the [Julia documentation](http://julia.readthedocs.org).

## Physical Constants

Some physical constants are represented in `YT`. They are available via the `YT.physical_constants` submodule, and are unitful quantities which can be used with other quantities and arrays:

In [ ]:
pc = YT.physical_constants
kb = pc.kboltz

In [ ]:
in_units(kb*sp["temperature"], "keV") # computing kT in kilo-electronvolts

Have a look inside [`YT.physical_constants`](https://github.com/jzuhone/YT.jl/blob/master/src/physical_constants.jl) to see which constants are implemented.

## Unit Symbols

Similarly, for convenience, many units implemented in `YT`, as well as prefixed versions where appropriate, have
corresponding `YTQuantities` which can be imported from the `YT.unit_symbols` module. They can then be multiplied by
`Real`s or `Array`s to generate `YTArray`s and `YTQuantities`:

In [ ]:
u = YT.unit_symbols

rand(5)*u.Msun

In [ ]:
3.0*u.kpc

## Equivalencies

Some physical quantities are directly related to other unitful quantities by a constant, but otherwise do not have the same units. To facilitate conversions between these quantities, `YT` implements a system of unit equivalencies (inspired by the [AstroPy implementation](http://docs.astropy.org/en/latest/units/equivalencies.html>).  

The possible unit equivalencies are:

* `"thermal"`: conversions between temperature and energy ($E = k_BT$)
* `"spectral"`: conversions between wavelength, frequency, and energy ($E = h\nu = hc/\lambda$, $c = \lambda\nu$)
* `"mass_energy"`: conversions between mass and energy ($E = mc^2$)
* `"lorentz"`: conversions between velocity and Lorentz factor ($\gamma = 1/\sqrt{1-(v/c)^2}$)
* `"schwarzschild"`: conversions between mass and Schwarzschild radius ($R_S = 2GM/c^2$)
* `"compton"`: conversions between mass and Compton wavelength ($\lambda = h/mc$)

The following unit equivalencies only apply under conditions applicable for an ideal gas with a constant mean molecular weight $\mu$ and ratio of specific heats $\gamma$:

* `"number_density"`: conversions between density and number density ($n = \rho/\mu{m_p}$)
* `"sound_speed"`: conversions between temperature and sound speed assuming an ideal gas ($c_s^2 = \gamma{k_BT}/\mu{m_p}$)

A `YTArray` or `YTQuantity` can be converted to an equivalent using the `to_equivalent()` method, where the unit and the equivalence name are provided as arguments:

In [ ]:
T = YTQuantity(1.0e8, "K")
to_equivalent(T, "keV", "thermal")

In [ ]:
ds2 = load("IsolatedGalaxy/galaxy0030/galaxy0030")

dd = AllData(ds2)

to_equivalent(dd["density"], "kpc^-3", "number_density")

In [ ]:
import YT.physical_constants: mp

to_equivalent(mp, "GeV", "mass_energy")

Some equivalencies take optional parameters, such as `"sound_speed"`, which allows you to change the mean molecular weight `mu` and ratio of specific heats `gamma`:

In [ ]:
kT = YTQuantity(4.0, "keV")
to_equivalent(kT, "km/s", "sound_speed", gamma=4./3., mu=0.5)

To list the available equivalencies for a given array or quantity, use the `list_equivalencies()` method:

In [ ]:
list_equivalencies(kT)

or to check if a specific equivalence exists for an array or quantity, use `has_equivalent()`:

In [ ]:
has_equivalent(kT, "spectral")

In [ ]:
has_equivalent(dd["density"], "compton")